In [148]:
!pip install pyspark

!pip install unidecode


You should consider upgrading via the '/home/fmenegolo/.asdf/installs/python/3.9.1/bin/python3.9 -m pip install --upgrade pip' command.
Reshimming asdf python...
You should consider upgrading via the '/home/fmenegolo/.asdf/installs/python/3.9.1/bin/python3.9 -m pip install --upgrade pip' command.
Reshimming asdf python...


In [149]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SQLContext
from datetime import datetime
from pyspark.sql.functions import col, substring, regexp_replace, expr, size, levenshtein, avg, when, format_number, sum, lower, trim, split, dense_rank
from pyspark.sql import functions as F
from pyspark.sql.window import Window
spark = SparkSession \
    .builder \
    .master("local[*]")\
    .appName("ELT Refined") \
    .getOrCreate()


In [150]:

path1 = f"/mnt/wsl/PHYSICALDRIVE2/Projects/Compasso/Azimute/Sprint_9/Desafio03_ETL/Tarefa_03/assets/Local/"
df_movies_IMDB_trusted = spark.read.parquet(path1 + 'movies_IMDB_trusted.parquet')
df_series_IMDB_trusted = spark.read.parquet(path1 + 'series_IMDB_trusted.parquet') 

path2 = f"/mnt/wsl/PHYSICALDRIVE2/Projects/Compasso/Azimute/Sprint_9/Desafio03_ETL/Tarefa_03/assets/TMDB/"
df_movies_TMDB_trusted = spark.read.parquet(path2 + 'movies_TMDB_trusted.parquet')
df_series_TMDB_trusted = spark.read.parquet(path2 + 'series_TMDB_trusted.parquet')


## 2. Normalização
#### 2.1. Cria tabela movies IMDB

In [151]:
df_movies_IMDB = df_movies_IMDB_trusted.select("id", "tituloPrincipal", "tituloOriginal", "anoLancamento", "tempoMinutos", "genero", "notaMedia", "numeroVotos")
df_movies_IMDB = df_movies_IMDB.dropDuplicates(['id'])  # Remover duplicatas pelo ID
df_movies_IMDB = df_movies_IMDB.withColumnRenamed("id", "movie_id")  # Renomear a coluna "id" para "movie_id"

#### 2.2. Cria tabela series IMDB

In [152]:
df_series_IMDB = df_series_IMDB_trusted.select("id", "tituloPrincipal", "tituloOriginal", "anoLancamento", "anoTermino", "tempoMinutos", "genero", "notaMedia", "numeroVotos")
df_series_IMDB = df_series_IMDB.dropDuplicates(['id'])  # Remover duplicatas pelo ID
df_series_IMDB = df_series_IMDB.withColumnRenamed("id", "seriesId")  # Renomear a coluna "id" para "movie_id"

## 3.Junção 

### 3.1 Unir DF dos FILMES

In [153]:
# Aplicar regexp_replace no dataframe df_movies_TMDB
df_movies_TMDB_titulos_datas = df_movies_TMDB_trusted.select(
    regexp_replace(col('title'), r"[^a-zA-Z0-9\s]", "").alias('title_replaced'),
    substring(col('release_date'), 1, 4).alias('releaseDate'),
    col('id'),
    col('genre_ids'),
    col('original_language'),
    col('popularity'),
    col('vote_average'),
    col('vote_count'),
    col('title'),
    col('release_date')
)

# Aplicar regexp_replace no dataframe df_movies_IMDB
df_movies_IMDB_titulos_datas = df_movies_IMDB.select(
    regexp_replace(col('tituloPrincipal'), r"[^a-zA-Z0-9\s]", "").alias('tituloPrincipal_replaced'),
    substring(col('anoLancamento'), 1, 4).alias('anoLancamento'),
    col('movie_id'),
    col('tituloOriginal'),
    col('tempoMinutos'),
    col('genero'),
    col('notaMedia'),
    col('numeroVotos')
)

# Remover a sentença "Final Chapter Part" do dataframe df_movies_IMDB
df_movies_IMDB_titulos_datas = df_movies_IMDB_titulos_datas.withColumn('tituloPrincipal_replaced', regexp_replace(col('tituloPrincipal_replaced'), 'Final Chapter Part', '')
)

# Realizar a comparação de similaridade entre os títulos dos dataframes e filtrar pelo ano de lançamento
df_movies_refined = df_movies_TMDB_titulos_datas.join(
    df_movies_IMDB_titulos_datas,
    expr(
        "(size(split(title_replaced, ' ')) = 1 AND " +
        "size(filter(split(title_replaced, ' '), x -> length(x) > 3 AND instr(tituloPrincipal_replaced, x) > 0)) >= 1) OR " +
        "(size(split(title_replaced, ' ')) = 2 AND " +
        "size(filter(split(title_replaced, ' '), x -> length(x) >= 1 AND instr(tituloPrincipal_replaced, x) > 0)) >= 2) OR " +
        "(size(split(title_replaced, ' ')) = 3 AND " +
        "size(filter(split(title_replaced, ' '), x -> length(x) > 4 AND instr(tituloPrincipal_replaced, x) > 0)) >= 2) OR " +
        "(size(split(title_replaced, ' ')) = 4 AND " +
        "size(filter(split(title_replaced, ' '), x -> length(x) > 3 AND instr(tituloPrincipal_replaced, x) > 0)) >= 3) OR " +
        "(size(split(title_replaced, ' ')) >= 5 AND " +
        "size(filter(split(title_replaced, ' '), x -> length(x) >= 3 AND instr(tituloPrincipal_replaced, x) > 0)) >= 3 AND " +
        "levenshtein(tituloPrincipal_replaced, title_replaced) <= 9)"
    ) &
    (df_movies_TMDB_titulos_datas['releaseDate'] == df_movies_IMDB_titulos_datas['anoLancamento']) &
    (col('tituloPrincipal_replaced').isNotNull()) &
    (col('tituloPrincipal_replaced') != '') &
    (col('title_replaced').isNotNull()) &
    (col('title_replaced') != ''),
    'left_outer'
)

In [154]:
# Calcular a soma de vote_count e numeroVotos (se não forem nulos)
df_movies_refined = df_movies_refined.withColumn('votos', col('vote_count') + col('numeroVotos'))

# Calcular a média de vote_average e notaMedia (se não forem nulos)
df_movies_refined = df_movies_refined.withColumn('nota_media', when(
    col('vote_count').isNotNull() & col('numeroVotos').isNotNull(),
    (col('vote_average') * col('vote_count') + col('notaMedia') * col('numeroVotos')) / col('votos')
).otherwise(
    when(col('vote_count').isNotNull(), col('vote_average')).otherwise(col('notaMedia'))
))

# Tratar valores nulos e arredondar a nota média ponderada para uma casa decimal
df_movies_refined = df_movies_refined.withColumn('votos', when(col('votos').isNull(), col('vote_count')).otherwise(col('votos')))
df_movies_refined = df_movies_refined.withColumn('nota_media', when(col('nota_media').isNull(), col('vote_average')).otherwise(col('nota_media')))
df_movies_refined = df_movies_refined.withColumn('nota_media', format_number(col('nota_media'), 1))

# Apagar as colunas indesejadas
df_movies_refined = df_movies_refined.drop('releaseDate', 'tituloPrincipal_replaced', 'anoLancamento', 'title_replaced','vote_average','vote_count','notaMedia','numeroVotos','tituloOriginal','genero')

# Remover caracteres especiais que não sejam letras acentuadas na coluna "title"
df_movies_refined = df_movies_refined.withColumn("title", regexp_replace(df_movies_refined["title"], "[^a-zA-ZÀ-ÿ, ]", ""))

# Selecione apenas as colunas desejadas
df_movies_refined = df_movies_refined.select('id','movie_id','genre_ids', 'title', 'tempoMinutos', 'release_date', 'original_language', 'popularity', 'nota_media', 'votos')

df_movies_refined = df_movies_refined.distinct()


In [155]:
# Exibir o resultado
df_movies_refined.show()

+-------+----------+--------------------+--------------------+------------+------------+-----------------+----------+----------+-----+
|     id|  movie_id|           genre_ids|               title|tempoMinutos|release_date|original_language|popularity|nota_media|votos|
+-------+----------+--------------------+--------------------+------------+------------+-----------------+----------+----------+-----+
| 349176|      null|   [28, 12, 35, 878]|Assassination Cla...|        null|  2015-03-21|               ja|    11.802|       7.3|  163|
|  16007| tt0758742|      [14, 9648, 53]|          Death Note|         126|  2006-06-17|               ja|    18.397|       7.5|31297|
| 359628| tt6475250|       [878, 35, 28]|Assassination Cla...|          92|  2016-03-25|               ja|    17.394|       5.6|  589|
| 420426| tt5979872|        [28, 14, 12]|              Bleach|         108|  2018-07-20|               ja|    15.138|       6.3|12418|
| 221731| tt3029558|    [28, 12, 18, 14]|Rurouni Kenshi

#### 3.2. Filtrar Series TMDB por Filme do TMDB (antes de unir DF SERIES)

In [156]:
# Aplicar regexp_replace no dataframe df_series_TMDB

df_series_TMDB_titulos_datas = df_series_TMDB_trusted.select(
    substring(col('first_air_date'), 1, 4).alias('releaseDate'),
    col('name_replaced'),
    col('id'),
    col('genre_ids'),
    col('original_language'),
    col('popularity'),
    col('vote_average'),
    col('vote_count'),
    col('name'),
    col('first_air_date')
)

# dataframe df_movies_TMDB
df_movies_TMDB_titulos_datas = df_movies_TMDB_titulos_datas.select(col('title_replaced'))

# Realizar a comparação de similaridade entre os títulos dos dataframes e filtrar pelo ano de lançamento
df_series_TMDB_Layer1 = df_series_TMDB_titulos_datas.join(
    df_movies_TMDB_titulos_datas,
    expr(
        "(size(split(name_replaced, ' ')) = 1 AND " +
        "size(filter(split(name_replaced, ' '), x -> length(x) > 3 AND instr(title_replaced, x) > 0)) >= 1) OR " +
        "(size(split(name_replaced, ' ')) = 2 AND " +
        "size(filter(split(name_replaced, ' '), x -> length(x) >= 1 AND instr(title_replaced, x) > 0)) >= 2) OR " +
        "(size(split(name_replaced, ' ')) = 3 AND " +
        "size(filter(split(name_replaced, ' '), x -> length(x) > 4 AND instr(title_replaced, x) > 0)) >= 2) OR " +
        "(size(split(name_replaced, ' ')) = 4 AND " +
        "size(filter(split(name_replaced, ' '), x -> length(x) > 3 AND instr(title_replaced, x) > 0)) >= 3) OR " +
        "(size(split(name_replaced, ' ')) >= 5 AND " +
        "size(filter(split(name_replaced, ' '), x -> length(x) >= 3 AND instr(title_replaced, x) > 0)) >= 3 AND " +
        "levenshtein(title_replaced, name_replaced) <= 8)"
    ) &
    (col('title_replaced').isNotNull()) &
    (col('title_replaced') != '') &
    (col('name_replaced').isNotNull()) &
    (col('name_replaced') != ''),
    'inner'
)

# Apagar as colunas indesejadas
df_series_TMDB_Layer1 = df_series_TMDB_Layer1.drop('title_replaced')

#### 3.1 Unir DF das SERIES

In [157]:
# Aplicar regexp_replace no dataframe df_movies_IMDB
df_series_IMDB_titulos_datas = df_series_IMDB.select(
    regexp_replace(col('tituloPrincipal'), r"[^a-zA-Z0-9\s]", "").alias('tituloPrincipal_replaced'),
    substring(col('anoLancamento'), 1, 4).alias('anoLancamento'),
    col('seriesId'),
    col('tituloPrincipal'),
    col('tempoMinutos'),
    col('anoTermino'),
    col('notaMedia'),
    col('numeroVotos')
)

# Realizar a comparação de similaridade entre os títulos dos dataframes e filtrar pelo ano de lançamento
df_series_refined = df_series_TMDB_Layer1.join(
    df_series_IMDB_titulos_datas,
    expr(
        "(size(split(name_replaced, ' ')) = 1 AND " +
        "size(filter(split(name_replaced, ' '), x -> length(x) > 3 AND instr(tituloPrincipal_replaced, x) > 0)) >= 1) OR " +
        "(size(split(name_replaced, ' ')) = 2 AND " +
        "size(filter(split(name_replaced, ' '), x -> length(x) >= 1 AND instr(tituloPrincipal_replaced, x) > 0)) >= 2) OR " +
        "(size(split(name_replaced, ' ')) = 3 AND " +
        "size(filter(split(name_replaced, ' '), x -> length(x) > 4 AND instr(tituloPrincipal_replaced, x) > 0)) >= 2) OR " +
        "(size(split(name_replaced, ' ')) = 4 AND " +
        "size(filter(split(name_replaced, ' '), x -> length(x) > 3 AND instr(tituloPrincipal_replaced, x) > 0)) >= 3) OR " +
        "(size(split(name_replaced, ' ')) >= 5 AND " +
        "size(filter(split(name_replaced, ' '), x -> length(x) >= 3 AND instr(tituloPrincipal_replaced, x) > 0)) >= 3 AND " +
        "levenshtein(tituloPrincipal_replaced, name_replaced) <= 9)"
    ) &
    (df_series_TMDB_Layer1['releaseDate'] == df_series_IMDB_titulos_datas['anoLancamento']) &
    (col('tituloPrincipal_replaced').isNotNull()) &
    (col('tituloPrincipal_replaced') != '') &
    (col('name_replaced').isNotNull()) &
    (col('name_replaced') != ''),
    'left_outer'
)


In [158]:
# Criar colunas com a soma dos votos e a média ponderada
df_series_refined = df_series_refined.withColumn('votos', col('vote_count') + col('numeroVotos'))

df_series_refined = df_series_refined.withColumn('nota_media', when(
    col('vote_count').isNotNull() & col('numeroVotos').isNotNull(),
    (col('vote_average') * col('vote_count') + col('notaMedia') * col('numeroVotos')) / col('votos')
).otherwise(
    when(col('vote_count').isNotNull(), col('vote_average')).otherwise(col('notaMedia'))
))

# Tratar valores nulos e arredondar a nota média ponderada para uma casa decimal
df_series_refined = df_series_refined.withColumn('votos', when(col('votos').isNull(), col('vote_count')).otherwise(col('votos')))
df_series_refined = df_series_refined.withColumn('nota_media', when(col('nota_media').isNull(), col('vote_average')).otherwise(col('nota_media')))
df_series_refined = df_series_refined.withColumn('nota_media', format_number(col('nota_media'), 1))

# Apagar as colunas indesejadas
df_series_refined = df_series_refined.drop('releaseDate', 'tituloPrincipal_replaced', 'releaseDate', 'name_replaced','vote_average','vote_count','notaMedia','numeroVotos','tituloPrincipal','anoLancamento')

# Remover caracteres especiais que não sejam letras acentuadas na coluna "name"
df_series_refined = df_series_refined.withColumn("name", regexp_replace(df_series_refined["name"], "[^a-zA-ZÀ-ÿ, ]", ""))

# Selecione apenas as colunas linhas desejadas
df_series_refined = df_series_refined.select('id','seriesId','genre_ids', 'name', 'tempoMinutos', 'first_air_date', 'anoTermino', 'original_language', 'popularity', 'nota_media', 'votos')
df_series_refined = df_series_refined.distinct()


In [159]:
# Exibir o resultado
df_series_refined.show()

+------+---------+--------------------+--------------------+------------+--------------+----------+-----------------+----------+----------+------+
|    id| seriesId|           genre_ids|                name|tempoMinutos|first_air_date|anoTermino|original_language|popularity|nota_media| votos|
+------+---------+--------------------+--------------------+------------+--------------+----------+-----------------+----------+----------+------+
| 66103|tt5603356|  [16, 80, 18, 9648]|        Ace Attorney|          24|    2016-04-02|      null|               ja|     28.72|       6.5|   459|
|105204|tt0367037|                [16]|New Fist of the N...|          58|    2003-07-04|      2004|               ja|     0.635|       6.8|   686|
|130728|tt4563516|            [16, 35]|DD Fist of the No...|          15|    2013-04-03|      null|               ja|      3.15|       6.7|    36|
|116087|     null|            [16, 18]|          Graduation|        null|    1995-01-21|      null|               ja| 

## 4. Cria tabela actors  

In [164]:

# Cria DF Actors
df_movies_actors = df_movies_IMDB_trusted.select("id", "personagem", "nomeArtista", "generoArtista", "anoNascimento", "anoFalecimento", "profissao", "titulosMaisConhecidos")
df_series_actors = df_series_IMDB_trusted.select("id", "personagem", "nomeArtista", "generoArtista", "anoNascimento", "anoFalecimento", "profissao", "titulosMaisConhecidos")

# Obter a lista de film_ids presentes na tabela de filmes selecionados
filme_ids = df_movies_refined.select("movie_id")
serie_ids = df_series_refined.select("seriesId")

# Realizar o join entre os DataFrames
df_movies_actors = df_movies_actors.join(filme_ids, df_movies_actors["id"] == filme_ids["movie_id"], "inner")
df_series_actors = df_series_actors.join(serie_ids, df_series_actors["id"] == serie_ids["seriesId"], "inner")

# Apaga coluna desnecessaria
df_movies_actors = df_movies_actors.drop("movie_id")
df_series_actors = df_series_actors.drop("seriesId")

# Unir df Atores de filmes e series
df_actors = df_movies_actors.unionAll(df_series_actors)

# Atualizar os dados da coluna com regex_replace pois dubladores atuam com mais de um pesonagem e os nomes estão unidos em camelCase
df_actors = df_actors.withColumn("personagem", regexp_replace(df_actors["personagem"], "([a-z])([A-Z])", "$1 $2"))
df_actors = df_actors.withColumn("nomeArtista", regexp_replace(df_actors["nomeArtista"], "([a-z])([A-Z])", "$1 $2"))

# Remover palavras repetidas dentro de cada célula da coluna "personagem"
df_actors = df_actors.withColumn("personagem", expr("regexp_replace(personagem, '\\b(\\w+)\\b\\s+\\1\\b', '$1')"))

# Remover conteúdo entre parênteses da coluna "personagem"
df_actors = df_actors.withColumn("personagem", regexp_replace(df_actors["personagem"], r'\([^()]*\)', ''))

# Substituir letras acentuadas por suas versões não acentuadas na coluna "personagem"
df_actors = df_actors.withColumn("personagem", regexp_replace(df_actors["personagem"], "[áàâãä]", "a"))
df_actors = df_actors.withColumn("personagem", regexp_replace(df_actors["personagem"], "[éèêë]", "e"))
df_actors = df_actors.withColumn("personagem", regexp_replace(df_actors["personagem"], "[íìîï]", "i"))
df_actors = df_actors.withColumn("personagem", regexp_replace(df_actors["personagem"], "[óòôõö]", "o"))
df_actors = df_actors.withColumn("personagem", regexp_replace(df_actors["personagem"], "[úùûü]", "u"))
df_actors = df_actors.withColumn("personagem", regexp_replace(df_actors["personagem"], "[ç]", "c"))

# Remover a palavra "Son" do início do nome na coluna "personagem"
df_actors = df_actors.withColumn("personagem", regexp_replace(df_actors["personagem"], "^Son ", ""))

# Atribua IDs únicos para artistas com nomes repetidos
window = Window.partitionBy("nomeArtista").orderBy("id")
df_actors = df_actors.withColumn("IdArtista", dense_rank().over(window))

# Atualizar o formato da coluna 'profissao'
df_actors = df_actors.withColumn('profissao', split(regexp_replace(df_actors['profissao'], ',\\s*', ','), ','))

# Atualizar o formato da coluna 'titulosMaisConhecidos'
df_actors = df_actors.withColumn('titulosMaisConhecidos', split(regexp_replace(df_actors['titulosMaisConhecidos'], ',\\s*', ','), ','))


In [165]:

count1 = df_movies_actors.count()
print("Número de linhas no resultado: M ", count1)

count2 = df_series_actors.count()
print("Número de linhas no resultado: S", count2)

count3 = df_actors.count()
print("Número de linhas no resultado: ", count3)

# Exibir o DataFrame atualizado
df_actors.show(1)
df_movies_actors.show(1)
df_series_actors.show(1)

Número de linhas no resultado: M  177
Número de linhas no resultado: S 258
Número de linhas no resultado:  435
+---------+--------------+-------------+-------------+-------------+--------------+--------------------+---------------------+---------+
|       id|    personagem|  nomeArtista|generoArtista|anoNascimento|anoFalecimento|           profissao|titulosMaisConhecidos|IdArtista|
+---------+--------------+-------------+-------------+-------------+--------------+--------------------+---------------------+---------+
|tt0421357|Alphonse Elric|Aaron Dismuke|        actor|         1992|          null|[actor, writer, s...| [tt0421357, tt560...|        1|
+---------+--------------+-------------+-------------+-------------+--------------+--------------------+---------------------+---------+
only showing top 1 row

+---------+------------+-----------+-------------+-------------+--------------+---------+---------------------+
|       id|  personagem|nomeArtista|generoArtista|anoNascimento|anoF

## 5. Cria Views  

In [ ]:
df_movies_refined.createOrReplaceTempView("movies")
df_series_refined.createOrReplaceTempView("series")
df_actors.createOrReplaceTempView("actors")

In [166]:
#df_movies_refined.show(3)
#df_series_refined.show(3)
df_actors.show()

+---------+--------------------+--------------------+-------------+-------------+--------------+--------------------+---------------------+---------+
|       id|          personagem|         nomeArtista|generoArtista|anoNascimento|anoFalecimento|           profissao|titulosMaisConhecidos|IdArtista|
+---------+--------------------+--------------------+-------------+-------------+--------------+--------------------+---------------------+---------+
|tt0421357|      Alphonse Elric|       Aaron Dismuke|        actor|         1992|          null|[actor, writer, s...| [tt0421357, tt560...|        1|
|tt0367037|               Seiji|         Adam Dudley|        actor|         null|          null|[actor, art_depar...| [tt0272880, tt006...|        1|
|tt0426719|         Bendot Kite|         Adam Hunter|        actor|         1982|          null|[music_department...| [tt0139774, tt028...|        1|
|tt0367037|                Tobi|     Akimitsu Takase|        actor|         1970|          null|    

23/06/11 17:35:41 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 83980409 ms exceeds timeout 120000 ms
23/06/11 17:35:41 WARN SparkContext: Killing executors is not supported by current scheduler.


## 5. Escreve os dados na camada Refined no S3 no formato parquet


In [ ]:
"""
# Obtém a data atual para criar os diretórios correspondentes
current_date = datetime.now()
ano = current_date.strftime("%Y")
mes = current_date.strftime("%m")
dia = current_date.strftime("%d")

# Define os caminhos de destino no S3 para os DataFrames no formato Parquet
path_ref_movies = "s3://data-lake-do-fabricio/Refined/Filmes/{ano}/{mes}/{dia}/"
path_ref_series = "s3://data-lake-do-fabricio/Refined/Series/{ano}/{mes}/{dia}/"
path_ref_actors = "s3://data-lake-do-fabricio/Refined/Actors/{ano}/{mes}/{dia}/"

# Salva os DataFrames no formato Parquet no S3
df_movies_refined.write.parquet(path_ref_movies.format(ano=ano, mes=mes, dia=dia))
df_series_refined.write.parquet(path_ref_series.format(ano=ano, mes=mes, dia=dia))
df_actors.write.parquet(path_ref_actors.format(ano=ano, mes=mes, dia=dia))
"""


'\n# Obtém a data atual para criar os diretórios correspondentes\ncurrent_date = datetime.now()\nano = current_date.strftime("%Y")\nmes = current_date.strftime("%m")\ndia = current_date.strftime("%d")\n\n# Define os caminhos de destino no S3 para os DataFrames no formato Parquet\npath_ref_movies = "s3://data-lake-do-fabricio/Refined/Filmes/{ano}/{mes}/{dia}/"\npath_ref_series = "s3://data-lake-do-fabricio/Refined/Series/{ano}/{mes}/{dia}/"\npath_ref_actors = "s3://data-lake-do-fabricio/Refined/Actors/{ano}/{mes}/{dia}/"\n\n# Salva os DataFrames no formato Parquet no S3\ndf_movies_refined.write.parquet(path_ref_movies.format(ano=ano, mes=mes, dia=dia))\ndf_series_refined.write.parquet(path_ref_series.format(ano=ano, mes=mes, dia=dia))\ndf_actors.write.parquet(path_ref_actors.format(ano=ano, mes=mes, dia=dia))\n'